In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
import os
from pathlib import Path

import paramiko
from scp import SCPClient

import getpass
import gc
import time

In [3]:
dir_github = '/media/rich/Home_Linux_partition/github_repos/'
import sys
sys.path.append(dir_github)

%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import server

In [4]:
# paramiko.util.log_to_file("paramiko.log")

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [5]:
# ## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
# dir_data_local = '/media/rich/bigSSD/tmp_data/'
# dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/data/test'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

# print(f'contents of    {dir_data_local}    will be copied to    {dir_data_MICROSCOPE}')

In [6]:
# remote_host_transfer = "transfer.rc.hms.harvard.edu"
# remote_host_compute = "o2.hms.harvard.edu"
# username = input('Username: ')
# pw = server.pw_encode(getpass.getpass(prompt='Password: '))

## Prepare `sftp` and `ssh` objects

In [11]:
# sftp = server.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
# sftp.connect(username=username, password=server.pw_decode(pw))

In [12]:
# # sftp.mkdir_safe(dir_data_remote)
# sftp.put_dir(dir_data_local, dir_data_MICROSCOPE)

------------
# ==== Start ====

## Prepare username + password

In [10]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = True

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username: rh183


## Prepare `sftp` and `ssh` objects

In [11]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

Exception ignored in: <function ssh_interface.__del__ at 0x7f91c42d9670>
Traceback (most recent call last):
  File "/media/rich/Home_Linux_partition/github_repos/basic_neural_processing_modules/server.py", line 395, in __del__
    self.ssh.close()
AttributeError: 'NoneType' object has no attribute 'close'


In [12]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

Exception ignored in: <function ssh_interface.__del__ at 0x7f91c42d9670>
Traceback (most recent call last):
  File "/media/rich/Home_Linux_partition/github_repos/basic_neural_processing_modules/server.py", line 395, in __del__
    self.ssh.close()
AttributeError: 'NoneType' object has no attribute 'close'


In [13]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

## Pull/update repo

In [14]:
ssh_c.send_receive('')
ssh_c.send('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm')
ssh_c.send_receive('git pull')

ssh_c.expect(str_success=f'[{username}', total_timeout=2);
ssh_c.send_receive('');

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.

remote: Total 9 (delta 2), reused 9 (delta 2), pack-reused 0
Unpacking objects:  66% (6/9)   
Unpacking objects: 100% (9/9), done.

From https://github.com/RichieHakim/face-rhythm
   734eb41..67fd9ec  dev        -> origin/dev

Updating 734eb41..67fd9ec

Fast-forward

 .../convert_nwb_to_dotCoordsNpy.py                 |    0
 batch_run/convert_nwb_to_npy/convert_nwb_to_npy.py |   51 +
 batch_run/{ => convert_nwb_to_npy}/dispatcher.py   |    0
 .../master_controller.ipynb                        |   54 +-
 batch_run/file_transfers_MasterController.ipynb    | 2368 ++------------------
 batch_run/{ => run_face_rhythm}/FRbatch.py         |    0
 batch_run/{ => run_face_rhythm}/FRbatch_CQTonly.py |    0
 .../batch_run_faceRhythm.ipynb                     |    0
 batch_run/run_face_rhythm/dispatcher.py            |  154 ++
 .../{ => run_face_rhythm}/environment_

## Activate environment

In [15]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv'

ssh_c.send_receive('')
ssh_c.send(f'conda activate {name_env}')
ssh_c.expect(str_success=f'[{username}', total_timeout=2);
ssh_c.send_receive('');

i/rich/virtual_envs/fr_venv/neurobio/sabatin 

hythm]$ 1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv) [rh183@login05 face-rh


## Prepare directories

In [22]:
import numpy as np

mouse_name = 'AEG22'
date = '2022_05_18'

name_slurm = mouse_name + '_' + date[-2:]

path_FRNWB        = Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/AEG21/2022_05_13/jobNum_0/batchRun/data/session_batch.nwb').as_posix()
dir_saveOutputs   = (Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/convert_nwb_to_npy') / mouse_name / date).as_posix()
path_FRNWB_remote = '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/AEG21/2022_05_13/jobNum_0/batchRun/data/session_batch.nwb'


name_job = 'jobNum_'


path_script_remote      = Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/batch_run/convert_nwb_to_npy/convert_nwb_to_npy.py').as_posix()  ## path to a copy of the remote_run_s2p.py file within the s2p_on_o2 repo on the server

path_dispatcher_local   = Path(r'/media/rich/Home_Linux_partition/github_repos/face-rhythm/batch_run/convert_nwb_to_npy/').resolve() / 'dispatcher.py'  ## path to the dispatcher.py file on local computer
path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

## Prepare commands

In [23]:
## Expectation for dispatch args: 
# path_selfScript = args[0] = path_dispatcher_remote
# dir_save = args[1] = dir_S2pOutput_remote
# path_script = args[2] = path_s2pScript_remote
# name_job = args[3] = name_job
# dir_fastDisk = args[4] = dir_fastDisk_remote
# name_slurm = args[5] = name_slurm
# dir_data = args[6] = dir_data_remote


prompt_snip = f'[{username}'

commands = {
#     'make_dir': f"mkdir -p {dir_data_remote}",
#     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
#     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
python3 \
{path_dispatcher_remote} \
{path_script_remote} \
{dir_saveOutputs} \
{path_FRNWB} \
{name_job} \
{name_slurm} \
{name_env}",
}
display(commands)

{'dispatch': 'python3 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/convert_nwb_to_npy/AEG22/2022_05_18/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/batch_run/convert_nwb_to_npy/convert_nwb_to_npy.py /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/convert_nwb_to_npy/AEG22/2022_05_18 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/AEG21/2022_05_13/jobNum_0/batchRun/data/session_batch.nwb jobNum_ AEG22_18 /n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv'}

## Upload `dispatcher.py` file

In [20]:
sftp.mkdir_p(Path(path_dispatcher_remote).parent.as_posix())
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Dispatch `remote_run_s2p.py`

In [21]:
ssh_c.send(commands['dispatch']);

ssh_c.expect(str_success=prompt_snip);

1/hms/neurobio/sabatini/rich/virtual_envs/fr_venvch.nwb jobNum_ AEG22_18 /n/data 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/batch_run/run_face_rhythm/convert_nwb_to_npy/convert_nwb_to_npy.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/convert_nwb_to_npy/AEG22/2022_05_18/convert_nwb_to_npy.py
Traceback (most recent call last):
  File "/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/convert_nwb_to_npy/AEG22/2022_05_18/dispatcher.py", line 80, in <module>
    shutil.copy2(path_script, str(Path(dir_save) / Path(path_script).name));
  File "/n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv/lib/python3.8/shutil.py", line 435, in copy2
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "/n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv/lib/python3.8/shutil.py", line 264, in copyfile
    with open(src, 'rb') as fsrc, open(dst, 'wb') as f

In [ ]:
/media/rich/Home_Linux_partition/github_repos/face-rhythm/batch_run/convert_nwb_to_npy

### check on job

In [181]:
ssh_c.send_receive('')
ssh_c.send(cmd=f'squeue -u {username} --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres');
ssh_c.expect(str_success=f"[{username}", total_timeout=10);

squeue -u rh183 --Format=jobid,name,partition,state,timeused,starttime,t 
imelimit,reasonlist,minmemory,mincpus,gres

JOBID               NAME                PARTITION           STATE               TIME                START_TIME          TIME_LIMIT          NODELIST(REASON)    MIN_MEMORY          MIN_CPUS            TRES_PER_NODE       
(/n/data1/hms/neurobio/sabatini/rich/virtual_envs/fr_venv) [rh183@login05 face-rh
hythm]$ 


In [340]:
## I think this code block caused an out of memory error on a few of my jobs... not sure why

# ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
# ssh_c.expect(str_success=f"[{username}", total_timeout=1);

In [27]:
ssh_c.send(cmd=f"cat {(Path(dir_S2pOutput_remote) / (name_job+'0') / 'print*').as_posix()}")
ssh_c.expect(str_success=f"RUN COMPLETE", recv_timeout=0.3, total_timeout=60*60*10, sleep_time=0.1, verbose=False);

print(f'RUN COMPLETE!!!     {time.ctime()}')

RUN COMPLETE!!!     Tue May 17 14:52:20 2022


## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs back from `data1` to `MICROSCOPE`

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `ssh` object

In [ ]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,
)

In [274]:
dir_S2pOutput_remote = dir_S2pOutput_remote  ## from above
dir_s2pOutput_MICROSCOPE = (Path(dir_data_MICROSCOPE) / 'suite2p_o2_output').as_posix()  ## from above
print(f'contents of    {dir_S2pOutput_remote}    will be copied to    {dir_s2pOutput_MICROSCOPE}')

contents of    /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14    will be copied to    /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output


In [275]:
commands = {
    'make_dir': f"mkdir -p {dir_s2pOutput_MICROSCOPE}",
    'copy_s2p': f"cp -r {(Path(dir_S2pOutput_remote)).as_posix()}/. {dir_s2pOutput_MICROSCOPE}",
}
display(commands)

{'make_dir': 'mkdir -p /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output',
 'copy_s2p': 'cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/AEG21/2022_05_14/. /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_14/suite2p_o2_output'}

In [276]:
ssh_t.send_receive(commands['make_dir']);

mkdir -p /n/files/Neurobio/MICROSCOPE/Ally/Mesoscop 
e/AEG21/2022_05_14/suite2p_o2_output



In [277]:
ssh_t.send(commands['copy_s2p'])
ssh_t.expect(str_success=f'[{username}', total_timeout=120, verbose=True);

(base) [rh183@transfer03 ~]$ 


## Clean up

In [ ]:
## delete password
del pw
gc.collect()
    
## close ssh
ssh_t.close()
del ssh_t
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs from `MICROSCOPE` to local machine

In [39]:
dir_s2pOutput_MICROSCOPE = dir_s2pOutput_MICROSCOPE  ## from above
dir_s2pOutput_local = str(Path(dir_data_local).resolve() / 'suite2p_o2_output')
print(f'contents of    {dir_s2pOutput_MICROSCOPE}    will be copied to    {dir_s2pOutput_local}')

contents of    /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output    will be copied to    /media/rich/bigSSD/tmp_data/suite2p_o2_output


In [57]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [40]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [41]:
sftp.get_dir(dir_s2pOutput_MICROSCOPE, dir_s2pOutput_local)

downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/params.json   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/params.json
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/print_log_53419173.log   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/print_log_53419173.log
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/batch_run_output.png   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/batch_run_output.png
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/log.txt   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/log.txt
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/sbatch_config.sh
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/remote_run_s2p.py   to   /medi

## Clean up

In [43]:
## delete password
del pw
gc.collect()

## close sftp
sftp.close()
del sftp

In [42]:
ssh_t.close()
ssh_c.close()